# Limpieza del Dataset

La primera actividad consiste en la limpieza del dataset, de manera que éste cumpla con los requerimientos del “Tidy Data”, es decir:

1. Cada renglón o fila del dataset corresponde a una observación o individuo.
2. Cada columna del dataset corresponde a una y sólo una variable.
3. No se mezcla información de naturaleza diferente.

En el caso del dataset entregado por el INE, estos principios, sobre todo el número 1, no se respetan ya que en nuestro caso, una observación/individuo es una (y sólo una) pregunta. El resultado de esta actividad es un dataset (almacenado en un archivo CSV) que contenga los siguientes campos:

- ID del registro
- Entidad de Origen
- Edad
- Género
- Identificación con algún grupo en situación de discriminación
- Tema de la pregunta
- Texto de la pregunta

Es decir, en los casos en los que una persona envió más de una pregunta, cada pregunta debe estar en su renglón con el resto de los datos de identificación.

# Processo

1. Se importan las librería necesarias para esta etapa.

In [33]:
import pandas as pd
from utils import save_df, DATASET
import re

2. Se lee el dataset y se cambian los valores de la columna 'Edad:' de 'str' a 'int'

In [34]:
crudo_df = pd.read_excel(DATASET)
crudo_df['Edad:'] = crudo_df['Edad:'].str.replace(' años', '').astype(int)

3. Se normalizan los nombres de las columnas.

In [37]:
crudo_df.rename(columns = {
        '¿Cuál es tu entidad de origen?': 'entidad', 
        'Edad:': 'edad',
        'Género:': 'genero',
        '¿Te identificas con alguno de los siguientes Grupos en Situación de Discriminación?': 'grupo_discriminacion',
        'Indica el tema de tu pregunta:': 'tema',
        'Escribe tu pregunta:': 'pregunta',
        '¿Deseas agregar otra pregunta?': 'extra',
        'Indica el tema de tu pregunta: (2)': 'tema2',
        'Escribe tu pregunta: (2)': 'pregunta2',
        '¿Deseas agregar otra pregunta? (2)': 'extra2',
        'Indica el tema de tu pregunta: (3)': 'tema3',
        'Escribe tu pregunta: (3)': 'pregunta3',
        '¿Deseas agregar otra pregunta? (3)': 'extra3',
        'Indica el tema de tu pregunta: (4)': 'tema4',
        'Escribe tu pregunta: (4)': 'pregunta4',
        '¿Deseas agregar otra pregunta? (4)': 'extra4',
        'Indica el tema de tu pregunta: (5)': 'tema5',
        'Escribe tu pregunta: (5)': 'pregunta5',
        '¿Deseas agregar otra pregunta? (5)': 'extra5',
        'Indica el tema de tu pregunta: (6)': 'tema6',
        'Escribe tu pregunta: (6)': 'pregunta6',
        'Fecha de entrada': 'fecha'
    }, inplace = True)

4. Se genera un nueva entrada si el usuario ingreso más de una pregunta.

Primero se crea una lista de DF con las siguientes entradas: 'entidad', 'edad', 'genero', 'grupo_discriminacion', 'tema','pregunta', 'fecha'. Cada DF contiene solo la primera pregunta de cada usuario

In [38]:
todo_preguntas = []
todo_preguntas.append(crudo_df[['entidad', 'edad', 'genero', 'grupo_discriminacion', 'tema','pregunta', 'fecha']])

Se generan las demás entradas con las demás preguntas ingresadas por el usuario.

In [39]:
#Asignación de los Archivos a la Lista
for idx in range(2, 6+1):
    todo_preguntas.append(crudo_df[['entidad', 'edad', 'genero', 'grupo_discriminacion', 'tema'+str(idx),'pregunta'+str(idx), 'fecha']].rename(columns={'tema'+str(idx): 'tema', 'pregunta'+str(idx): 'pregunta'}))

Se concatenan todos los DF en un solo DF y se eliminan las entradas que contengan NA (que significa que el usuario no ingreso pregunta X)

In [40]:
#Concatenación de los Elementos de la Lista a un DataFrame
limpio_df = pd.concat(todo_preguntas)
limpio_df = limpio_df.dropna()

5. Se normaliza algunas columnas:
    1. Los usuarios que no pertenezcan a ningún grupo de discriminación pasa de 'Selecciona' a 'N/A'.
    2. La columna de 'fecha' pasa de 'str' a 'timestap'.
    3. Se eliminar las entradas exactamente iguales.
    4. Remover saltos de linea.

In [55]:
limpio_df['grupo_discriminacion'] = limpio_df['grupo_discriminacion'].str.replace('Selecciona','No Aplica')
limpio_df['fecha'] = limpio_df['fecha'].replace({'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'}, regex=True)
limpio_df['fecha'] = pd.to_datetime(limpio_df['fecha'], format='%d %m, %Y %I:%M %p')
limpio_df['pregunta'] = limpio_df['pregunta'].replace(r'\n', ' ', regex=True) # Este paso puede generar dos espacios
limpio_df.drop_duplicates(inplace=True)

6. Se guarda el DF

In [53]:
save_df(limpio_df, 0, index=False)